# Automate Animated GIF Extraction

Project takes an animated GIF *source-2.gif* and automatically disassebles it into multiple jpg files which are then displayed in order on a set of four 5-tile LIFX tilechains.


In [5]:
#Main Program for Convert Single Image to Tiles

# Full running function with all dependencies
#imports RGB to HSBK conversion function from LIFX LAN library
import os
from lifxlan import LifxLAN
from lifxlan.utils import RGBtoHSBK
from pylifxtiles import tiles
from pylifxtiles import actions
from matplotlib import image
from PIL import Image
from PIL import GifImagePlugin


# modify this variable to the name of the specific LIFX Tilechain as shown in the LIFX app

source_folder = './images/'
target_folder = './images/target/'
animated_gif = "./images/IMG_1424.GIF"

def main():
    lan = LifxLAN()
    tilechain_lights = lan.get_tilechain_lights()
    print(len(tilechain_lights))
    if len(tilechain_lights) != 0:
        for tile in tilechain_lights:
            if tile.get_label()  ==  'T1':
                print(tile.get_label())
                T1 = tile
            if tile.get_label()  =='T2':
                print(tile.get_label())
                T2 = tile
            if tile.get_label()  == 'T3':
                print(tile.get_label())
                T3 = tile
            if tile.get_label()  == 'T4':
                print(tile.get_label())
                T4 = tile
    tc_list = [T1, T2, T3, T4]
    try: 
        #extract images from animated gif first
        #set first file number to '0'
        file_num = '000'
        increment = 0
        imageObject = Image.open("./images/IMG_1424.GIF")
        for frame in range(0,imageObject.n_frames):
            #set new filename
            for new_file in range(150):
                if len(str(increment)) == 1:
                    file_name= file_num[:2]+str(increment)
                elif len(str(increment)) == 2:
                    file_name= file_num[:1]+str(increment)
                else:
                    file_name = str(increment)
            increment += 1
            my_file = str(file_name)+'image'
            #seek out specific frame in animated gif
            imageObject.seek(frame)
            #save frame to png file
            new_frame = imageObject.convert('RGB')
            new_frame.save('./images/target/'+my_file+".jpg")
            #increment file_num by 1
        #create target_list from 
        my_images = os.listdir(target_folder)
        my_images.sort()
        for walk in range(10):
            for i in my_images:
                target_file = (target_folder+i)
                display_image(target_file, tc_list)
    except KeyboardInterrupt:
        print("Done.")




#combined function

# resize image and force a new shape and save to disk
def display_image(image_to_display, tilechain_list):
    # load the image
    my_image = Image.open(image_to_display)
    # report the size of the image
    #print(my_image.size)
    # resize image and ignore original aspect ratio
    img_resized = my_image.resize((40,32))
    #changing the file extension from jpg to png changes output brightness. You might need to play with this. 
    img_resized.save('./images/resized_image.jpg')
    data = image.imread('./images/resized_image.jpg')
    target_tcs = []
    for row in data:
        temp_row = []
        for pixel in row:
            temp_row.append(RGBtoHSBK(pixel))
        target_tcs.append(temp_row)
    tcsplit = tiles.split_tilechains(target_tcs)
    tilechain_list[0].set_tilechain_colors(tiles.split_combined_matrix(tcsplit[0]),rapid=True)
    tilechain_list[1].set_tilechain_colors(tiles.split_combined_matrix(tcsplit[1]),rapid=True)
    tilechain_list[2].set_tilechain_colors(tiles.split_combined_matrix(tcsplit[2]),rapid=True)
    tilechain_list[3].set_tilechain_colors(tiles.split_combined_matrix(tcsplit[3]),rapid=True)
        
if __name__ == "__main__":
    main()

21
T2
T4
T3
T1
